# Gallery Example: M/M/1 Tandem Multiclass Network

This example demonstrates a 2-queue tandem network with multiple classes:
- **Arrivals**: Exponential (separate Poisson process per class)
- **Service**: Exponential (class-dependent rates)
- **Servers**: 1 server per queue
- **Scheduling**: FCFS
- **Classes**: 2 job classes with different arrival and service rates

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_mm1_tandem_multiclass():
    """Create M[2]/M[2]/1 tandem multiclass queueing network"""
    model = Network('M[2]/M[2]/1 -> -/M[2]/1')
    
    # Block 1: nodes
    source = Source(model, 'Source')
    queue1 = Queue(model, 'Queue1', SchedStrategy.FCFS)
    queue2 = Queue(model, 'Queue2', SchedStrategy.FCFS)
    sink = Sink(model, 'mySink')
    
    # Block 2: classes
    oclass1 = OpenClass(model, 'myClass1')
    source.set_arrival(oclass1, Exp(1))
    queue1.set_service(oclass1, Exp(4))
    queue2.set_service(oclass1, Exp(6))
    
    oclass2 = OpenClass(model, 'myClass2')
    source.set_arrival(oclass2, Exp(0.5))
    queue1.set_service(oclass2, Exp(2))
    queue2.set_service(oclass2, Exp(6))
    
    # Block 3: topology
    P = model.init_routing_matrix()
    P.add_route(oclass1, source, queue1, 1.0)
    P.add_route(oclass1, queue1, queue2, 1.0)
    P.add_route(oclass1, queue2, sink, 1.0)
    P.add_route(oclass2, source, queue1, 1.0)
    P.add_route(oclass2, queue1, queue2, 1.0)
    P.add_route(oclass2, queue2, sink, 1.0)
    model.link(P)
    
    return model

# Create the model
model = gallery_mm1_tandem_multiclass()

## About Multiclass Tandem Networks

This model has two classes with different characteristics:

**Class 1**:
- Arrival rate: λ₁ = 1
- Service rate at Queue1: μ₁₁ = 4
- Service rate at Queue2: μ₁₂ = 6

**Class 2**:
- Arrival rate: λ₂ = 0.5
- Service rate at Queue1: μ₂₁ = 2
- Service rate at Queue2: μ₂₂ = 6

With FCFS scheduling, both classes share the server capacity but have different service requirements.

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)